In [18]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date,datetime, timedelta
import investpy
import time
from stage import fullPrint

In [19]:
SectorDict = {
    "S&P 500 Energy":"XLE",
    "S&P 500 Information Technology":"XLK",
    "S&P 500 Telecom Services":"XLC",
    "S&P 500 Consumer Staples":"XLP",
    "S&P 500 Health Care":"XLV",
    "S&P 500 Consumer Discretionary":"XLY",
    "S&P 500 Industrials":"XLI",
    "S&P 500 Utilities":"XLU",
    "S&P 500 Materials":"XLB",
    "S&P 500 Financials":"XLF",
    "S&P 500 Real Estate":"XLRE"}
now = datetime.now()
sectorDfList = []
for sector in SectorDict.keys():
    df = investpy.indices.get_index_historical_data(sector, 'United States', '01/01/2010', '01/01/2022', as_json=False, order='ascending', interval='Weekly')
    df['sector'] = SectorDict[sector]
    sectorDfList.append(df)
    sectorDfList[-1].to_pickle("stockData/sectorCharts/"+SectorDict[sector]+".pkl")
    time.sleep(5)

In [26]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spDf = investpy.indices.get_index_historical_data("S&P 500", 'United States', '01/01/2002', '01/01/2022', as_json=False, order='ascending', interval='Weekly')
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index]['Close']/df.iloc[index-30]['Close']
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['MA30']/df.iloc[index-1]['MA30']
i = 0
for df in sectorDfList:
    df['MA30'] = df['Close'].rolling(window=30).apply(lambda x: np.sum(x)/30)
    df['MA30Slope'] = df.apply(lambda x: calculateSlope(df,df.index.get_loc(x.name)),axis=1)
    df['Volume'] = df['Volume'].replace(to_replace=0, method='ffill')
    df['percent'] = df.apply(lambda x: product(df,df.index.get_loc(x.name)),axis=1)
    spDf['percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    df['RS'] = df['percent'] - spDf['percent']
    df.drop(['Open','High','Low','Close','Volume','percent','Currency'],axis=1)
    df.to_pickle("stockData/sectorCharts/"+df['sector'].iat[0]+".pkl")
    i += 1

In [35]:
goodSectorDf = sectorDfList[0].drop(['Open','High','Low','Close','Currency','MA30','MA30Slope','RS','sector','percent','Volume'],axis=1)

In [36]:
goodSectorDf.index = sectorDfList[0].index
fullPrint(sectorDfList[1][['MA30Slope','RS']])

            MA30Slope        RS
Date                           
2010-01-03   0.000000 -1.210070
2010-01-10   0.000000 -1.233167
2010-01-17   0.000000 -1.188116
2010-01-24   0.000000 -1.197954
2010-01-31   0.000000 -1.212779
2010-02-07   0.000000 -1.143697
2010-02-14   0.000000 -1.132661
2010-02-21   0.000000 -1.118494
2010-02-28   0.000000 -1.126890
2010-03-07   0.000000 -1.145306
2010-03-14   0.000000 -1.130364
2010-03-21   0.000000 -1.133789
2010-03-28   0.000000 -1.159091
2010-04-04   0.000000 -1.145426
2010-04-11   0.000000 -1.115913
2010-04-18   0.000000 -1.165553
2010-04-25   0.000000 -1.157509
2010-05-02   0.000000 -1.036762
2010-05-09   0.000000 -1.044131
2010-05-16   0.000000 -1.007494
2010-05-23   0.000000 -1.051361
2010-05-30   0.000000 -0.995866
2010-06-06   0.000000 -0.998281
2010-06-13   0.000000 -1.023942
2010-06-20   0.000000 -0.986505
2010-06-27   0.000000 -0.924592
2010-07-04   0.000000 -0.974286
2010-07-11   0.000000 -0.965904
2010-07-18   0.000000 -0.978854
2010-07-

In [37]:
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf['BadSectors'] = [[] for _ in range(len(goodSectorDf))]
for date in sectorDfList[0].index:
    listOfSector = []
    listOfBadSector = []
    for df in sectorDfList:
        try:
            if df.at[date,'MA30Slope'] > 1 or df.loc[date,'RS'] > 0.1:
                if df.at[date,'Close'] > df.at[date,'MA30']:
                    listOfSector.append(df.at[date,'sector'])
            if (df.at[date,'MA30Slope'] < 0.995 and df.loc[date,'RS'] < 0) or df.at[date, 'MA30Slope'] < 0.975:
                if df.at[date,'Close'] < df.at[date,'MA30']:
                    listOfBadSector.append(df.at[date,'sector'])
        except: 
            print(df.iloc[-1]['sector'])
    goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    goodSectorDf.at[date,'BadSectors'] = set(listOfBadSector)

In [38]:
fullPrint(goodSectorDf)
goodSectorDf.to_pickle('stockData/goodSector.pkl')

                                                      Sectors  \
Date                                                            
2010-01-03                                                 {}   
2010-01-10                                                 {}   
2010-01-17                                                 {}   
2010-01-24                                                 {}   
2010-01-31                                                 {}   
2010-02-07                                                 {}   
2010-02-14                                                 {}   
2010-02-21                                                 {}   
2010-02-28                                                 {}   
2010-03-07                                                 {}   
2010-03-14                                                 {}   
2010-03-21                                                 {}   
2010-03-28                                                 {}   
2010-04-04               